Importing Libs

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

Importing the data

In [ ]:
df_results = pd.read_csv('F1/results.csv')
df_drivers = pd.read_csv('F1/drivers_updates.csv')
df_constructors = pd.read_csv('F1/constructors.csv')
df_status = pd.read_csv('F1/status.csv')
df_races = pd.read_csv('F1/races.csv')
df_drivers_standings = pd.read_csv('F1/driver_standings.csv')

In [ ]:
df_races['raceId'] = df_races['raceId'].astype(int)

Merging the Datasets and Analysing the data

In [ ]:
merge1 = pd.merge(df_results, df_races, on='raceId')
merge2 = pd.merge(merge1, df_drivers, on ='driverId')
merge3 = pd.merge(merge2, df_constructors, on = 'constructorId')
df_complete = pd.merge(merge3, df_status, on = 'statusId')


In [ ]:
pd.set_option("display.max_columns",None)
df_complete.head()

In [ ]:
df_complete.isnull().sum()

In [ ]:
df_complete.isna().sum()

In [ ]:
df_complete.info()

In [ ]:
df_complete.describe().T

In [ ]:
df_complete.head()

Dropping Unwanted Columns and Renaming the remaining ones

In [ ]:
df_complete = df_complete.drop(columns=['url', 'url_x', 'url_y', 'fastestLapTime', 'time_y', 'fp1_date', 'round', 'circuitId',
                                        'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time',
                                        'sprint_date', 'sprint_time'])

In [ ]:
df_complete.columns

In [ ]:
col_name = {'number_x':'number','milliseconds':'timetaken_in_millisec', 'time_x': 'time', 'name_x': 'GrandPrixName',
            'number_y':'driverNum','code':'driverCode','nationality_x':'driverNationality','name_y':'constructorName',
            'nationality_y': 'constructorNationality', 'name': 'constructorName', 'fastestLapSpeed': 'maxSpeed'}

df_complete.rename(columns=col_name,inplace=True)
df_complete.head()

In [ ]:
## Merging Fist and Last name
df_complete['driverName'] = df_complete['forename']+' '+df_complete['surname']
df_complete = df_complete.drop(columns = ['forename','surname'])

In [ ]:
df_complete.head()

In [ ]:
df_complete.info()

Changing Date Type

In [ ]:
df_complete['dob'] = df_complete['dob'].str.strip()
df_complete['dob'] = pd.to_datetime(df_complete['dob'], dayfirst=True, errors='coerce')

## Handling Null values and unformatted date to convert it to datetime
df_complete['driverDeath'] = df_complete['driverDeath'].str.strip()
df_complete['driverDeath'] = df_complete['driverDeath'].replace('\\N', np.nan)
df_complete['driverDeath'] = pd.to_datetime(df_complete['driverDeath'], errors='coerce', dayfirst=True)


In [ ]:
df_complete['dob'].info()

In [ ]:
## Checking if the convertion worked
df_complete[df_complete['driverDeath'].notna()].head()

In [ ]:
## Adding Column: driverAge
df_complete['driverAge'] = np.where(
    df_complete['driverDeath'].notna(),
    (df_complete['driverDeath'] - df_complete['dob']).dt.days / 365,
    (datetime.today() - df_complete['dob']).dt.days / 365
)

df_complete['driverAge'] = df_complete['driverAge'].fillna(0)

df_complete['driverAge'] = df_complete['driverAge'].round().astype(int)

In [ ]:
## Changing the numeric data type
change_data_type = ['number', 'position', 'time', 'timetaken_in_millisec', 'fastestLap', 'rank', 'maxSpeed']
for i in change_data_type:
    df_complete[i] = pd.to_numeric(df_complete[i], errors='coerce')

In [ ]:
df_complete.head()

In [ ]:
## Checking null values
df_complete.isnull().sum() / len(df_complete) * 100

In [ ]:
df_complete[['maxSpeed', 'driverAge']].mean()

In [ ]:
df_complete.info()

In [ ]:
# df_filter = df_complete[(df_complete['driver_name'] == 'Ayrton Senna') & (df_complete['position'] == 1)]
# df_filter.head()

In [ ]:
df_complete[['rank','fastestLap']] = df_complete[['rank','fastestLap']].fillna(0)
df_complete['timetaken_in_millisec'] = df_complete['timetaken_in_millisec'].fillna(df_complete['timetaken_in_millisec'].mean())
df_complete['maxSpeed']= df_complete['maxSpeed'].fillna(df_complete['maxSpeed'].mean())
df_complete['number'] = df_complete['number'].fillna(0)

In [ ]:
df_complete.describe().T

In [ ]:
# # Filter numeric columns only
# numeric_df = df_complete.select_dtypes(include=['number'])

# # Calculate the quantiles and IQR for numeric data
# Q1 = numeric_df.quantile(0.25)
# Q3 = numeric_df.quantile(0.75)
# IQR = Q3 - Q1

# # Apply the IQR filter
# df_complete = df_complete[~((numeric_df < (Q1 - 1.5 * IQR)) | (numeric_df > (Q3 + 1.5 * IQR))).any(axis=1)]

# df_complete.head()


Changing Nationality to Country Name

In [ ]:
# Creating the variable to list the the Nationalities in the main Dataframe
nationalities = list(df_complete['driverNationality'].drop_duplicates())

In [ ]:
nationalities[:10]

Importing the Dataset with the Countries names

In [ ]:
df_countries = pd.read_csv('F1/Nationalities.csv')
df_countries.head()

In [ ]:
df_countries.info()

Updating the main Dataframe

In [ ]:
nationality_map = dict(zip(df_countries['Nationality'], df_countries['Country']))
df_complete['driverNationality'] = df_complete['driverNationality'].map(nationality_map).fillna(df_complete['driverNationality'])
df_complete.head()

Saving the DataFrame as a CSV file

In [ ]:
df_complete.to_csv('F1DataExploration.csv', index=False)